In [1]:
# get all imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, log_loss # if you're on mac using miniconda miniforge3 install miniforge::compilers to fix issue with this module
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import numpy as np
import os

In [3]:
# Define data transforms from the Ai_Competition github
data_transforms = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(512),        # Randomly crop the image to 512x512
            transforms.RandomHorizontalFlip(),        # Randomly flip the image horizontally
            transforms.ToTensor(),                    # Convert the image to a tensor
            transforms.Normalize([0.485, 0.456, 0.406], # Normalize with standard values
                                 [0.229, 0.224, 0.225]) # Mean and Std for RGB channels
        ]),
        'val': transforms.Compose([
            transforms.Resize(512),                    # Resize the image to 512x512
            transforms.CenterCrop(512),                # Crop from the center
            transforms.ToTensor(),                     # Convert the image to a tensor
            transforms.Normalize([0.485, 0.456, 0.406], # Normalize with standard values
                                 [0.229, 0.224, 0.225]) # Mean and Std for RGB channels
        ]),
    }

In [4]:
data_dir = "AI_Competition"
batch_size=32
image_datasets = {x: datasets.ImageFolder(data_dir,
    data_transforms[x])
    for x in ['train', 'val']}

dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size,
    shuffle=True, num_workers=4)
    for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [5]:
# Load a pre-trained ResNet18 model
model = models.resnet18(pretrained=True)

# Get the input dimensions of the last layer (fully connected layer)
num_ftrs = model.fc.in_features

# Replace the last layer with a new one that has `num_classes` outputs
model.fc = nn.Linear(num_ftrs, len(class_names))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/Users/kevincrothers/miniforge3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/kevincrothers/miniforge3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [46]:
num_epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9) 
best_model_wts = model.state_dict()  # Store the best model weights
best_acc = 0.0  # Track the best validation accuracy

for epoch in range(num_epochs):
    print(f'Epoch {epoch}/{num_epochs - 1}')
    print('-' * 10)
    
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # Set the model to training mode
        else:
            model.eval()   # Set the model to evaluation mode
        
        running_loss = 0.0  # Track the loss
        running_corrects = 0  # Track the number of correct predictions
        
        for inputs, labels in dataloaders[phase]:
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to the GPU/CPU
            
            optimizer.zero_grad()  # Clear the gradients
            
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)  # Forward pass
                _, preds = torch.max(outputs, 1)  # Get the class with the highest score
                loss = criterion(outputs, labels)  # Calculate the loss
                
                if phase == 'train':
                    loss.backward()  # Backpropagate the loss
                    optimizer.step()  # Update the weights
            
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        
        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]
        
        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        
        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = model.state_dict()

print(f'Best val Acc: {best_acc:.4f}')

model.load_state_dict(best_model_wts)

Epoch 0/9
----------


In [ ]:
# Evaluate the model on the validation set
model.eval()  # Set model to evaluation mode
all_labels = []
all_preds = []
all_probs = []
running_loss = 0.0

with torch.no_grad():
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Calculate the loss
        
        running_loss += loss.item() * inputs.size(0)
        
        _, preds = torch.max(outputs, 1)  # Get the class with the highest score
        probs = torch.nn.functional.softmax(outputs, dim=1)  # Get probabilities
        
        all_labels.extend(labels.cpu().numpy())  # Store true labels
        all_preds.extend(preds.cpu().numpy())    # Store predicted labels
        all_probs.extend(probs.cpu().numpy())    # Store probabilities

avg_loss = running_loss / len(dataloader.dataset)  # Calculate average loss

# Calculate overall metrics
accuracy = accuracy_score(all_labels, all_preds)
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
logloss = log_loss(all_labels, all_probs)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Log Loss: {logloss:.4f}')

In [ ]:
# Finally save the best model!
torch.save(model.state_dict(), "best_model.pth") 